In [119]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')
client.database_names()
db = client.data_diggers
collection = db.training_data


## import the package and data

In [120]:
from pprint import pprint
import numpy
import pymongo
import pandas as pd
from sklearn import linear_model
import sklearn

filter_ = {
                            'house_price':{'$exists':'true'},
#                               'total_jobs':{'$exists':'true'},
#                               'school_number':{'$exists':'true'},
                              'unemployment':{'$exists':'true'},
                              'traffic_noise':{'$exists':'true'},
#                               'hospitals_number':{'$exists':'true'},
                              'population':{'$exists':'true'},
                              'GVA':{'$exists':'true'},
#                               'stations_number':{'$exists':'true'},
                              'happiness':{'$exists':'true'},
#                               'pubs_number':{'$exists':'true'},
                              'road_traffic_2016':{'$exists':'true'},
                             }

all_found = collection.find(filter_)


# read data to pandas
data = pd.DataFrame(list(all_found))
# print(data[0:10])

# create target variable
target = data['happiness']
# print(target)


# remove happiness from data
data = data.drop(['happiness'], axis=1)

data = data.fillna(0)
# create target variable
# print(data[0:10])

# remove some columns
data = data.drop(['_id', 'city_name', 'latitude', 'longitude', 'region', 'sample', 'uk_region'], axis=1)

# print(data[0:82])



# Insert the colum and normalize

In [121]:
# normalization by z-score 
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
def scaleColumns(df):
    cols_to_scale = list(df)
    for col in cols_to_scale:
        df[col] = pd.DataFrame(scaler.fit_transform(pd.DataFrame(df[col])),columns=[col])
    return df

In [122]:
data['hospitals_number_per_person'] = data['hospitals_number']/data['population']
data['pubs_number_per_person'] = data['pubs_number']/data['population']
data['stations_number_per_person'] = data['stations_number']/data['population']
data['school_number_per_person'] = data['school_number']/data['population']
data['total_jobs_per_person'] = data['total_jobs']/data['population']
data['number_of_universities_per_person'] = data['number_of_universities']/data['population']


# remove some columns
# data = data.drop(['hospitals_number', 'pubs_number', 'stations_number', 'school_number', 'total_jobs', 'number_of_universities'], axis=1)

# remove some columns
# data = data.drop(['FrostDay_perYear', 'Rainfall_perMon', 'SummerDay_ave_temperature', 'SummerNight_ave_temperature', 'Sunshine_perMon', 'WinterDay_ave_temperature', 'WinterNight_ave_temperature'], axis=1)


# normalization by z-score 
data = scaleColumns(data)
print(data[0:5])


   FrostDay_perYear       GVA  Rainfall_perMon  SummerDay_ave_temperature  \
0          0.265005 -0.604379        -1.187813                   0.307294   
1          0.265005 -0.604379        -1.187813                   0.307294   
2          0.071818 -0.479550         1.317848                  -1.253429   
3          0.071818 -0.716725         1.317848                  -1.253429   
4          0.071818 -0.716725         1.317848                  -1.253429   

   SummerNight_ave_temperature  Sunshine_perMon  WinterDay_ave_temperature  \
0                    -0.477843        -0.180513                  -0.047880   
1                    -0.477843        -0.180513                  -0.047880   
2                    -0.628893        -1.236739                  -1.238173   
3                    -0.628893        -1.236739                  -1.238173   
4                    -0.628893        -1.236739                  -1.238173   

   WinterNight_ave_temperature  hospitals_number  house_price  \
0  

In [123]:
data_pandas = data
# convert both to Numpy array representation
data = data.as_matrix()
target = target.as_matrix()

# Linear regression : Lasso

In [124]:
# run lasso for feature selection
clf = linear_model.Lasso(alpha=0.0005)
clf.fit(data, target)

alphas = numpy.geomspace(0.00001, 0.1, num=100)
print(alphas)

error = []
for i in alphas:
    
    mean_error = []
    
#     cross validation
    kf = sklearn.model_selection.KFold(4, shuffle=True)
    for train_index, test_index in kf.split(data):
        X_train, X_test = data[train_index], data[test_index]
        Y_train, Y_test = target[train_index], target[test_index]
        
#         train model
        clf = linear_model.Lasso(alpha=i)
        clf.fit(X_train, Y_train)
        
#         error
        mean_error.append(sklearn.metrics.mean_squared_error(Y_test, clf.predict(X_test)))
    
#     error
    error.append(numpy.mean(mean_error))

    
print(error)
print(error.index(min(error)))
print(alphas[error.index(min(error))])

# alpha
alpha = alphas[error.index(min(error))]

clf = linear_model.Lasso(alpha=alpha)
clf.fit(data, target)


weights = clf.coef_
print(weights.tolist())
print(clf.intercept_)

/usr/local/anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[  1.00000000e-05   1.09749877e-05   1.20450354e-05   1.32194115e-05
   1.45082878e-05   1.59228279e-05   1.74752840e-05   1.91791026e-05
   2.10490414e-05   2.31012970e-05   2.53536449e-05   2.78255940e-05
   3.05385551e-05   3.35160265e-05   3.67837977e-05   4.03701726e-05
   4.43062146e-05   4.86260158e-05   5.33669923e-05   5.85702082e-05
   6.42807312e-05   7.05480231e-05   7.74263683e-05   8.49753436e-05
   9.32603347e-05   1.02353102e-04   1.12332403e-04   1.23284674e-04
   1.35304777e-04   1.48496826e-04   1.62975083e-04   1.78864953e-04
   1.96304065e-04   2.15443469e-04   2.36448941e-04   2.59502421e-04
   2.84803587e-04   3.12571585e-04   3.43046929e-04   3.76493581e-04
   4.13201240e-04   4.53487851e-04   4.97702356e-04   5.46227722e-04
   5.99484250e-04   6.57933225e-04   7.22080902e-04   7.92482898e-04
   8.69749003e-04   9.54548457e-04   1.04761575e-03   1.14975700e-03
   1.26185688e-03   1.38488637e-03   1.51991108e-03   1.66810054e-03
   1.83073828e-03   2.00923300e-03

In [125]:
# select the features
boundary = 0.0001

selectedFeatures = []
data_deleted = data_pandas

for i in range(len(weights)):
    if abs(weights[i])> boundary:
        selectedFeatures.append(data_pandas.columns.values[i])
    else:
        data_deleted = data_deleted.drop(data_pandas.columns.values[i],axis=1)

In [126]:
# convert both to Numpy array representation
data_d = data_deleted.as_matrix()
clf = linear_model.Lasso(alpha=alpha)
clf.fit(data_d, target)
weights = clf.coef_
print(weights.tolist())
print(clf.intercept_)
data_deleted.columns

[-0.0019481288152185558, -0.014841331847123187, 0.018164382570711794, -0.030284999643427284, -0.000909616936167845, 0.007302827360101339, -0.029277961724133904, 0.008681538170107708, -0.02730125937111946, 0.008202475985350247, -0.014230753927213834, -0.014127719183873202]
7.32226190476


Index(['FrostDay_perYear', 'GVA', 'Rainfall_perMon', 'hospitals_number',
       'road_traffic_2016', 'school_number', 'stations_number',
       'traffic_noise', 'unemployment', 'pubs_number_per_person',
       'stations_number_per_person', 'number_of_universities_per_person'],
      dtype='object')

In [127]:
from pprint import pprint

# save features to dictionary
values = weights.tolist()
keys = data_deleted.columns

count = 0
valueFeatures = {}
for key in keys:
    valueFeatures[key] = values[count]
    count += 1


sortedFeatures = sorted(valueFeatures, key=lambda dict_key: abs(valueFeatures[dict_key]))
print("The rank of importance: less to most")  
pprint(sortedFeatures)
valueFeatures

The rank of importance: less to most
['road_traffic_2016',
 'FrostDay_perYear',
 'school_number',
 'pubs_number_per_person',
 'traffic_noise',
 'number_of_universities_per_person',
 'stations_number_per_person',
 'GVA',
 'Rainfall_perMon',
 'unemployment',
 'stations_number',
 'hospitals_number']


{'FrostDay_perYear': -0.0019481288152185558,
 'GVA': -0.014841331847123187,
 'Rainfall_perMon': 0.018164382570711794,
 'hospitals_number': -0.030284999643427284,
 'number_of_universities_per_person': -0.014127719183873202,
 'pubs_number_per_person': 0.008202475985350247,
 'road_traffic_2016': -0.000909616936167845,
 'school_number': 0.007302827360101339,
 'stations_number': -0.029277961724133904,
 'stations_number_per_person': -0.014230753927213834,
 'traffic_noise': 0.008681538170107708,
 'unemployment': -0.02730125937111946}

## Randomforest

In [132]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=3, random_state=0)
regr.fit(data, target)
importance_rank_randomForest = regr.feature_importances_.tolist()
keys = list(data_pandas.columns)
randomForest_result = {}
for i in range(len(importance_rank_randomForest)):
    randomForest_result[keys[i]] = importance_rank_randomForest[i]
sortedFeatures = sorted(randomForest_result, key=lambda dict_key: abs(randomForest_result[dict_key]))
print("The rank of importance: less to most")  
pprint(sortedFeatures)
collection = db.factorsRank
collection.drop()
collection.insert_one(randomForest_result)
collection.find_one()

The rank of importance: less to most
['GVA',
 'SummerDay_ave_temperature',
 'SummerNight_ave_temperature',
 'Sunshine_perMon',
 'WinterNight_ave_temperature',
 'house_price',
 'pubs_number',
 'total_jobs_per_person',
 'number_of_universities',
 'total_jobs',
 'Rainfall_perMon',
 'school_number_per_person',
 'stations_number_per_person',
 'traffic_noise',
 'population',
 'number_of_universities_per_person',
 'WinterDay_ave_temperature',
 'FrostDay_perYear',
 'pubs_number_per_person',
 'road_traffic_2015',
 'unemployment',
 'hospitals_number',
 'hospitals_number_per_person',
 'road_traffic_2016',
 'school_number',
 'stations_number']


{'FrostDay_perYear': 0.033045029672154634,
 'GVA': 0.0,
 'Rainfall_perMon': 0.00431350549475259,
 'SummerDay_ave_temperature': 0.0,
 'SummerNight_ave_temperature': 0.0,
 'Sunshine_perMon': 0.0,
 'WinterDay_ave_temperature': 0.0328025577282421,
 'WinterNight_ave_temperature': 0.0,
 '_id': ObjectId('5a4d6a8f7c4c7b78d807aa2b'),
 'hospitals_number': 0.0963585313356559,
 'hospitals_number_per_person': 0.1122355017094772,
 'house_price': 0.0,
 'number_of_universities': 0.002081415584475558,
 'number_of_universities_per_person': 0.025579038082267767,
 'population': 0.01773388467915592,
 'pubs_number': 4.621981803250472e-05,
 'pubs_number_per_person': 0.05236912566887063,
 'road_traffic_2015': 0.07044313944400817,
 'road_traffic_2016': 0.12059161823794644,
 'school_number': 0.14239702755686234,
 'school_number_per_person': 0.005290481558341714,
 'stations_number': 0.1725803581912307,
 'stations_number_per_person': 0.006260976034281876,
 'total_jobs': 0.004009982598353862,
 'total_jobs_per_pers

## pearsonr

In [129]:
from scipy.stats import pearsonr
keys = list(data_pandas.columns)
for i in range(len(data[0,:])):
    p = pearsonr(data[:,i],target)
    print("the %s pearsonr correlation coefficient is %f, p-value is %f " % (keys[i],p[0],p[1]))

the FrostDay_perYear pearsonr correlation coefficient is -0.084784, p-value is 0.443201 
the GVA pearsonr correlation coefficient is 0.063358, p-value is 0.566944 
the Rainfall_perMon pearsonr correlation coefficient is 0.117249, p-value is 0.288157 
the SummerDay_ave_temperature pearsonr correlation coefficient is 0.043415, p-value is 0.694961 
the SummerNight_ave_temperature pearsonr correlation coefficient is 0.124612, p-value is 0.258735 
the Sunshine_perMon pearsonr correlation coefficient is 0.253310, p-value is 0.020078 
the WinterDay_ave_temperature pearsonr correlation coefficient is 0.070056, p-value is 0.526584 
the WinterNight_ave_temperature pearsonr correlation coefficient is 0.125220, p-value is 0.256402 
the hospitals_number pearsonr correlation coefficient is -0.501243, p-value is 0.000001 
the house_price pearsonr correlation coefficient is 0.179193, p-value is 0.102900 
the number_of_universities pearsonr correlation coefficient is -0.364889, p-value is 0.000643 
the

In [130]:
for i in range(len(data[0,:])):
    for j in range(len(data[0,:])):
        p = pearsonr(data[:,i],data[:,j])
        print("the %s and %s pearsonr correlation coefficient is %f, p-value is %f " % (keys[i],keys[j],p[0],p[1]))

the FrostDay_perYear and FrostDay_perYear pearsonr correlation coefficient is 1.000000, p-value is 0.000000 
the FrostDay_perYear and GVA pearsonr correlation coefficient is -0.128693, p-value is 0.243341 
the FrostDay_perYear and Rainfall_perMon pearsonr correlation coefficient is 0.064880, p-value is 0.557642 
the FrostDay_perYear and SummerDay_ave_temperature pearsonr correlation coefficient is -0.315916, p-value is 0.003418 
the FrostDay_perYear and SummerNight_ave_temperature pearsonr correlation coefficient is -0.810783, p-value is 0.000000 
the FrostDay_perYear and Sunshine_perMon pearsonr correlation coefficient is -0.388298, p-value is 0.000262 
the FrostDay_perYear and WinterDay_ave_temperature pearsonr correlation coefficient is -0.081161, p-value is 0.462997 
the FrostDay_perYear and WinterNight_ave_temperature pearsonr correlation coefficient is -0.926359, p-value is 0.000000 
the FrostDay_perYear and hospitals_number pearsonr correlation coefficient is 0.182725, p-value i

the WinterDay_ave_temperature and WinterDay_ave_temperature pearsonr correlation coefficient is 1.000000, p-value is 0.000000 
the WinterDay_ave_temperature and WinterNight_ave_temperature pearsonr correlation coefficient is 0.421439, p-value is 0.000065 
the WinterDay_ave_temperature and hospitals_number pearsonr correlation coefficient is -0.361855, p-value is 0.000719 
the WinterDay_ave_temperature and house_price pearsonr correlation coefficient is 0.760998, p-value is 0.000000 
the WinterDay_ave_temperature and number_of_universities pearsonr correlation coefficient is -0.376450, p-value is 0.000417 
the WinterDay_ave_temperature and population pearsonr correlation coefficient is -0.345143, p-value is 0.001303 
the WinterDay_ave_temperature and pubs_number pearsonr correlation coefficient is -0.684927, p-value is 0.000000 
the WinterDay_ave_temperature and road_traffic_2015 pearsonr correlation coefficient is -0.219184, p-value is 0.045158 
the WinterDay_ave_temperature and road_t